In [1]:
# Import necessary libraries
import yfinance as yf
import pandas as pd

# Define the ticker symbols for assets
tickers = {
    "Apple": "AAPL",
    "NVIDIA": "NVDA",
    "Microsoft": "MSFT",
    "Alphabet (Google)": "GOOG",
    "Amazon": "AMZN"
}

# Manually set market cap for Gold and Bitcoin (in trillions USD)
# Using https://companiesmarketcap.com/assets-by-market-cap/
gold_market_cap = 18.439  # In trillions
bitcoin_market_cap = 2.011  # In trillion

# Manually set the circulating supply of Bitcoin (in units) using https://www.coingecko.com/en/coins/bitcoin
btc_circulating_supply = 19420000  # Bitcoin 

# Convert to USD
gold_market_cap_usd = gold_market_cap * 1e12
bitcoin_market_cap_usd = bitcoin_market_cap * 1e12

# Function to fetch stock data using Yahoo Finance
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "Asset": ticker,
        "Market Cap (USD)": info.get("marketCap")
    }

# Aggregate data for stocks
data = []
for name, ticker in tickers.items():
    try:
        data.append(fetch_stock_data(ticker))
    except Exception as e:
        print(f"Failed to fetch data for {name} ({ticker}): {e}")

# Add Gold and Bitcoin manually
data.append({"Asset": "GOLD", "Market Cap (USD)": gold_market_cap_usd})
data.append({"Asset": "BTC", "Market Cap (USD)": bitcoin_market_cap_usd})

# Create a DataFrame
df = pd.DataFrame(data)

# Sort by market cap in descending order
df = df.sort_values(by="Market Cap (USD)", ascending=False)

# Format Market Cap in trillions (with $ sign)
df["Market Cap (USD)"] = df["Market Cap (USD)"].apply(lambda x: f"${x / 1e12:.3f}T")

# Set Asset column as the index
df.set_index("Asset", inplace=True)

# Display the updated DataFrame
df

,Market Cap (USD)
Asset,
GOLD,$18.439T
AAPL,$3.457T
MSFT,$3.231T
NVDA,$2.900T
AMZN,$2.475T
GOOG,$2.361T
BTC,$2.011T


In [2]:
# Calculate minimum BTC price to overtake each asset
btc_prices = []
for asset, row in df.iterrows():
    if asset == "BTC":
        continue
    # Parse the market cap value back to a float (removing the dollar sign and 'T')
    market_cap_target = float(row["Market Cap (USD)"].replace("$", "").replace("T", "")) * 1e12
    btc_price = market_cap_target / btc_circulating_supply
    btc_prices.append({"Target Asset": asset, "Minimum BTC Price (USD)": btc_price})

# Create a DataFrame for the results and display it
btc_price_df = pd.DataFrame(btc_prices)

# Format the Minimum BTC Price (USD) column as currency
btc_price_df["Minimum BTC Price (USD)"] = btc_price_df["Minimum BTC Price (USD)"].apply(
    lambda x: "${:,.2f}".format(x)
)

# Sort by target asset market cap in ascending order
btc_price_df = btc_price_df.sort_values(by="Minimum BTC Price (USD)", ascending=True).reset_index(drop=True)

# Display the minimum BTC prices
btc_price_df

,Target Asset,Minimum BTC Price (USD)
0,GOOG,"$121,575.70"
1,AMZN,"$127,445.93"
2,NVDA,"$149,330.59"
3,MSFT,"$166,374.87"
4,AAPL,"$178,012.36"
5,GOLD,"$949,485.07"
